In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
filename = "KRW-XRP-1m-small"

df_origin = pd.read_csv(f"../data/{filename}-train.csv")

In [4]:
# 필요한 열 선택
df = df_origin[['candle_date_time_kst', 'opening_price', 'high_price', 'low_price', 'trade_price', 'candle_acc_trade_volume']]

# datetime 형식으로 변환
df['candle_date_time_kst'] = pd.to_datetime(df['candle_date_time_kst'])
df.set_index('candle_date_time_kst', inplace=True)

# 결측치 제거
df.dropna(inplace=True)



scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df)

import joblib
joblib.dump(scaler, '../util/scaler.pkl')

# DataFrame으로 변환
scaled_df = pd.DataFrame(scaled_df, index=df.index, columns=df.columns)

/tmp/ipykernel_1577797/4058343266.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['candle_date_time_kst'] = pd.to_datetime(df['candle_date_time_kst'])
/tmp/ipykernel_1577797/4058343266.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [ ]:
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data) - seq_length):
        x = data.iloc[i:(i+seq_length)].values
        y = data.iloc[i+seq_length]['trade_price']
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

seq_length = 60  # 예: 60분(1시간) 동안의 데이터를 사용하여 다음 값을 예측
X, y = create_sequences(scaled_df, seq_length)

In [ ]:
np.save(f'../preprocessed/{filename}-X.npy', X)
np.save(f'../preprocessed/{filename}-y.npy', y)